In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re

# Data counts (image counts)

In [ ]:
# read in the csv
import pandas as pd
import numpy as np

df = pd.read_csv('/bask/homes/f/fspo1218/amber/data/gbif_macro_data/gbif_macro/datacount.csv')
df.head()

In [ ]:
# create a histogram of the counts
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(15, 3))

ax[0].hist(df['count'], bins=20)
ax[0].set_title('Counts of species images downloaded (990 species total)')
ax[0].set_xlabel('Number of images collected')
ax[0].set_ylabel('Counts')


ax[1].hist(df.loc[df['count'] < 100, 'count'], bins=20)
ax[1].set_title('Counts of species images downloaded (where number < 100)')
ax[1].set_xlabel('Number of images collected')
ax[1].set_ylabel('Counts')

plt.show()


df = pd.read_csv('/bask/projects/v/vjgo8416-amber/data/gbif-species-trainer-AMI-fork/gbif_images/try_wrapper')

In [ ]:
df = pd.read_csv('/bask/projects/v/vjgo8416-amber/data/gbif-species-trainer-AMI-fork/gbif_images/try_wrapper/data_statistics.csv')
df.shape

In [ ]:
# create a histogram of the counts
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(15, 3))

ax[0].hist(df['image_count'], bins=20)
ax[0].set_title('Counts of species images downloaded (' + str(df.shape[0]) + ' species total)')
ax[0].set_xlabel('Number of images collected')
ax[0].set_ylabel('Counts')


ax[1].hist(df.loc[df['image_count'] < 100, 'image_count'], bins=20)
ax[1].set_title('Counts of species images downloaded (where number < 100)')
ax[1].set_xlabel('Number of images collected')
ax[1].set_ylabel('Counts')

plt.show()

In [ ]:
df = pd.read_csv('/bask/projects/v/vjgo8416-amber/projects/on_device_classifier/02_model_training/pytorch/myfile.csv')
df.head()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


acc_type = ("Top 10", "Top 3", "Top 1")
accuracies = {
    'TF Lite': (sum(df['TFLite_top10'])/df.shape[0] * 100, sum(df['TFLite_top3'])/df.shape[0] * 100, sum(df['TFLite_top1'])/df.shape[0] * 100),
    'Pytorch': (sum(df['Pytorch_top10'])/df.shape[0] * 100, sum(df['Pytorch_top3'])/df.shape[0] * 100, sum(df['Pytorch_top1'])/df.shape[0] * 100),
}

x = np.arange(len(acc_type))  # the label locations
width = 0.25  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(layout='constrained')

for attribute, measurement in accuracies.items():
    print_meas = [round(x, 2) for x in measurement]
    offset = width * multiplier
    rects = ax.bar(x + offset, print_meas, width, label=attribute)
    ax.bar_label(rects, padding=3)
    multiplier += 1

ax.set_ylabel('Accuracy')
ax.set_xlabel('Accuracy Type')
ax.set_title('Accuracy of Species Classification Models')
ax.set_xticks([x + 0.125 for x in list(range(0, len(acc_type)))], acc_type)
ax.legend(loc='upper left', ncols=3)
ax.set_ylim(0, 100)

plt.show()

In [ ]:
def calc_acc(true_col, pred_col):
    temp = [1 if df[true_col][i] == df[pred_col][i] else 0 for i in range(df.shape[0])] 
    temp = sum(temp)/df.shape[0] * 100
    return(temp)

In [ ]:
acc_type = ("Species", "Genus", "Family")

accuracies = {
    'TF Lite': (calc_acc('True_label', 'TFLite_prediction'), calc_acc('True_genus', 'TFLite_genus'), calc_acc('True_family', 'TFLite_family')),
    'Pytorch': (calc_acc('True_label', 'Pytorch_prediction'), calc_acc('True_genus', 'Pytorch_genus'), calc_acc('True_family', 'Pytorch_family')),
}

x = np.arange(len(acc_type))  # the label locations
width = 0.25  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(layout='constrained')

for attribute, measurement in accuracies.items():
    print_meas = [round(x, 2) for x in measurement]
    offset = width * multiplier
    rects = ax.bar(x + offset, print_meas, width, label=attribute)
    ax.bar_label(rects, padding=3)
    multiplier += 1

ax.set_ylabel('Accuracy (%)')
ax.set_xlabel('Taxonomic Level')
ax.set_title('Accuracy of Classification at Taxonomic Levels')
ax.set_xticks([x + 0.125 for x in list(range(0, len(acc_type)))], acc_type)
ax.legend(loc='upper left', ncols=3)
ax.set_ylim(0, 100)

plt.show()

In [ ]:
df.loc[df['True_label_index'] == 0, ]

In [ ]:
df3 = pd.crosstab(df["TFLite_prediction"], df["True_label"], dropna=True, normalize="index")
df3 = df3.fillna(0)


# Force the df to be square
i = list(set(list(df["TFLite_prediction"]) + list(df["True_label"])))
df3 = df3.reindex(index=i, columns=i, fill_value=0)

df3.head()

In [ ]:
import operator

fam_dict = pd.Series(list(df.True_family) + list(df.True_family), index=list(df.True_label)+list(df.TFLite_prediction)).to_dict()
gen_dict = pd.Series(list(df.True_genus) + list(df.True_genus),index=list(df.True_label)+list(df.TFLite_prediction)).to_dict()

In [ ]:
fam_col = sns.color_palette("husl", len(set(fam_dict.values())))
gen_col = sns.color_palette("husl", len(set(gen_dict.values())))

fam_col = dict(zip(list(set(fam_dict.values())), [tuple(int(c*255) for c in cs) for cs in fam_col]))
gen_col = dict(zip(list(set(gen_dict.values())), [tuple(int(c*255) for c in cs) for cs in gen_col]))

# #colors = [tuple(t / 255 for t in x) for x in rgb]

fam_col = [fam_col[fam_dict[x]] for x in list(df3.columns)]
gen_col = [gen_col[gen_dict[x]] for x in list(df3.columns)]

fam_col = [tuple(round(t / 255, 2) for t in x) for x in fam_col]
gen_col = [tuple(round(t / 255, 2) for t in x) for x in gen_col]

fam_col[0:10]

In [ ]:
from scipy.cluster.hierarchy import linkage
import matplotlib

link = linkage(df3.transpose()) 

genus_colors = pd.DataFrame(data={'genus': gen_col }) 
family_colors = pd.DataFrame(data={'family': fam_col }) 

sns.set(font_scale=0.25)

sns.clustermap(df3, row_linkage=link, col_linkage=link, row_colors=genus_colors, col_colors=family_colors, 
               norm=matplotlib.colors.LogNorm(), figsize=(20,15))


plt.show()